In [97]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb -O /tmp/chrome.deb
!sudo apt-get update
!sudo -E apt-get install -y /tmp/chrome.deb
!pip install chromedriver-autoinstaller selenium

import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

--2023-12-07 13:23:30--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.250.179.78, 2a00:1450:4007:80e::200e
Connecting to dl.google.com (dl.google.com)|142.250.179.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 104956912 (100M) [application/x-debian-package]
Saving to: ‘/tmp/chrome.deb’

/tmp/chrome.deb     100%[===================>] 100.09M  19.7MB/s    in 5.0s    

2023-12-07 13:23:36 (20.0 MB/s) - ‘/tmp/chrome.deb’ saved [104956912/104956912]

Get:1 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]      
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:4 https://dl.google.com/linux/chrome/deb stable/main amd64 Packages [1,076 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]        
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backp

'/opt/mamba/lib/python3.10/site-packages/chromedriver_autoinstaller/120/chromedriver'

In [98]:
!pip install webdriver_manager

In [99]:
! pip install selenium

In [183]:
import pandas

In [100]:
import selenium
from webdriver_manager.chrome import ChromeDriverManager

path_to_web_driver = ChromeDriverManager().install()

In [101]:
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--user-agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:120.0) Gecko/20100101 Firefox/120.0'") 

In [102]:
from selenium.webdriver.chrome.service import Service
service = Service(executable_path=path_to_web_driver)

browser = webdriver.Chrome(service=service,
                           options=chrome_options)

In [278]:
browser.get('https://www.vinted.fr/catalog?search_text=tshirt+noir+homme')
time.sleep(1)

#browser.get_screenshot_as_file('screeniebefore.png')

try:  # handling cookies pop-upabs
    Cookies = browser.find_element(By.ID, 'onetrust-accept-btn-handler')
    Cookies.click()
    time.sleep(1)  # let's be respectful !
except NoSuchElementException:
    pass

In [279]:
n=20
firstscrapping = {}

for i in range(1,n+1) :
    XPATH = '/html/body/main/div/section/div/div[2]/section/div/div[2]/div[15]/div/div['+str(i)+']'
    x = browser.find_element(By.XPATH, XPATH)
    classx = x.get_attribute('class')
    if len(classx) > 15 :
        continue
    XPATH = '/html/body/main/div/section/div/div[2]/section/div/div[2]/div[15]/div/div['+str(i)+']/div/div/div/div[2]/a'
    x = browser.find_element(By.XPATH, XPATH)
    firstscrapping[i] = x.get_attribute('href')

In [288]:
firstscrapping2={}
for i in firstscrapping.keys() :
    browser.get(firstscrapping[i])
    time.sleep(1)
    try:  # handling cookies pop-upabs
        Cookies = browser.find_element(By.ID, 'onetrust-accept-btn-handler')
        Cookies.click()
        time.sleep(1)  # let's be respectful !
    except NoSuchElementException:
        pass
    firstscrapping2[i]={}
    XPATH = '/html/body/main/div/section/div/div[2]/main'
    infos = browser.find_element(By.XPATH, XPATH)
    x = infos.text.find('\n')
    y = infos.text.find('€')
    firstscrapping2[i]['prix'] = infos.text[x+1:y+1]
    x = infos.text.find('MARQUE')
    if x == -1 :
        firstscrapping2[i]['marque'] = float('nan')
    else :
        y = infos.text.find('BRAND')
        firstscrapping2[i]['marque'] = infos.text[x+6:y-1]
    x = infos.text.find('TAILLE')
    if x == -1 :
        firstscrapping2[i]['taille'] = float('nan')
    else :
        y = infos.text.find('SIZE')
        firstscrapping2[i]['taille'] = infos.text[x+6:y-1]

In [291]:
firstscrapping2[1].values()

dict_values(['1,00 €', 'ADIDAS', 'S'])

In [293]:
df = pandas.DataFrame.from_dict(firstscrapping2, orient = 'index', columns = firstscrapping2[1].keys())

In [299]:
df = df.transpose(copy=True)

In [300]:
df

,1,2,3,4,5,6,8,9,10,11,12,13,14,15,16,17,18,19,20
prix,"1,00 €","1,00 €","1,00 €","1,00 €","1,00 €","4,00 €","1,00 €","1,00 €","2,00 €","2,50 €","1,00 €","3,00 €","2,00 €","1,00 €","2,00 €","1,00 €","1,00 €","2,00 €","2,50 €"
marque,ADIDAS,KIPSTA,DECATHLON,NaN,NaN,LE COQ SPORTIF,FRUIT OF THE LOOM,LES TEEESH,THE NORTH FACE,PUMA,KIABI,SHEIN,JE NE SAIS PAS,HARRY POTTER,ZARA,FILA,NaN,WED'ZE,REDSKINS
taille,S,M,L,XXXL,XXL,M,XXL,M,XS,XL,L,XS,XS,M,M,L,S,M,XL


In [235]:
x = price.text.find('\n')
y = price.text.find('€')
price.text[x+1:y+1]

'1,00 €'

In [201]:
browser.get_screenshot_as_file('workin2.png')

True

In [287]:
tutu = r'12\n345'
pos = tutu.find('dfsd')
print(pos)
tutu[pos+1]

-1


'1'